In [ ]:
import os

# Test the exact path our code will use
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
work_folder = os.path.join(desktop_path, "Facial Deviation Work")

print(f"Target folder: {work_folder}")
print(f"Desktop exists: {os.path.exists(desktop_path)}")
print(f"Desktop writable: {os.access(desktop_path, os.W_OK)}")

try:
    os.makedirs(work_folder, exist_ok=True)
    test_file = os.path.join(work_folder, "test.txt")
    with open(test_file, 'w') as f:
        f.write("test")
    os.remove(test_file)
    print("Success! Can create folder and files")
except Exception as e:
    print(f" Error: {e}")

In [ ]:
import cv2
import numpy as np
import pandas as pd
import subprocess
import os
import tempfile
import time
from datetime import datetime
import shutil
import math
import json

class RealTimeFacialDeviationTracker:
    def __init__(self, openface_path="/Users/alibekdadajonov/openface_install/external_libs/openFace/OpenFace"):
        """
        Initialize the Real-Time Facial Deviation Tracking System with OpenFace UI Features
        Based on: Research paper from Frontiers in Psychology (2020)
        Using RMSD Formula: sqrt(Σ(d(pf, pf+a)²) / N) where N = 68 facial landmarks
        """
        print("OpenFace Real-Time Facial Deviation Tracking System (RMSD-Based)")
        print("=" * 70)
        print("Based on: Frontiers in Psychology Research Paper (2020)")
        print("RMSD Formula: sqrt(Σ(d(pf, pf+a)²) / N)")
        print("Where d(pf, pf+a) = sqrt((x_pf+a - x_pf)² + (y_pf+a - y_pf)²)")
        print("Features: 3D Head Pose Cube + Eye Gaze Tracking + Normalized RMSD + DEBUG MODE")
        print("=" * 70)
        
        self.openface_path = openface_path
        self.feature_extraction = os.path.join(openface_path, "build", "bin", "FeatureExtraction")
        if not os.path.exists(self.feature_extraction):
            self.feature_extraction = os.path.join(openface_path, "exe", "FeatureExtraction")
        
        self.baseline_landmarks = None
        self.baseline_face_stats = None
        self.baseline_normalized_landmarks = None  # NEW: Store normalized baseline
        self.deviation_data = []
        self.detailed_calculations = []
        self.accumulated_rmsd = 0.0
        self.frame_count = 0
        self.debug_mode = True
        
        # Use system temp directory instead of creating our own
        self.temp_dir = tempfile.mkdtemp(prefix="openface_")
        print(f"Using temporary directory: {self.temp_dir}")

    def normalize_landmarks_to_face_size(self, landmarks):
        """
        Normalize landmarks relative to face bounding box to handle position/scale variations
        This is the KEY FIX for the coordinate system issues
        """
        if landmarks is None or len(landmarks) == 0:
            return None
            
        # Calculate face bounding box
        x_coords = landmarks[:, 0]
        y_coords = landmarks[:, 1]
        
        face_left = np.min(x_coords)
        face_right = np.max(x_coords)
        face_top = np.min(y_coords)
        face_bottom = np.max(y_coords)
        
        face_width = face_right - face_left
        face_height = face_bottom - face_top
        face_center_x = (face_left + face_right) / 2
        face_center_y = (face_top + face_bottom) / 2
        
        # Normalize to unit coordinate system (0-1 range)
        normalized_landmarks = np.zeros_like(landmarks)
        
        if face_width > 0 and face_height > 0:
            # Center landmarks around face center, then normalize by face dimensions
            normalized_landmarks[:, 0] = (landmarks[:, 0] - face_center_x) / face_width
            normalized_landmarks[:, 1] = (landmarks[:, 1] - face_center_y) / face_height
        
        return normalized_landmarks

    def setup_baseline(self, num_frames=30):
        """Capture baseline neutral expression with proper normalization"""
        print(f"\n✓ Setting up baseline from {num_frames} frames...")
        print("Please maintain a neutral expression and stay in the same position")
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            raise Exception("Could not open webcam")
        
        # Set consistent camera parameters
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        baseline_landmarks_list = []
        baseline_normalized_list = []
        baseline_face_stats_list = []
        
        for i in range(num_frames):
            ret, frame = cap.read()
            if not ret:
                continue
                
            # Save frame temporarily
            temp_image = os.path.join(self.temp_dir, f"baseline_{i}.jpg")
            cv2.imwrite(temp_image, frame)
            
            # Process with OpenFace
            face_data = self.extract_landmarks_from_image(temp_image)
            if face_data is not None and 'landmarks_2d' in face_data:
                landmarks = face_data['landmarks_2d']
                
                # Store both raw and normalized landmarks
                baseline_landmarks_list.append(landmarks)
                
                # Normalize landmarks for consistent comparison
                normalized_landmarks = self.normalize_landmarks_to_face_size(landmarks)
                if normalized_landmarks is not None:
                    baseline_normalized_list.append(normalized_landmarks)
                
                # Collect face statistics
                face_stats = self.check_face_consistency(landmarks)
                if face_stats:
                    baseline_face_stats_list.append(face_stats)
                
                print(f"✓ Baseline frame {i+1}/{num_frames} processed")
            
            # Show preview
            cv2.putText(frame, f"Baseline Setup: {i+1}/{num_frames}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, "Keep still and maintain neutral expression", (10, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
            cv2.imshow('Baseline Setup', frame)
            cv2.waitKey(100)
        
        cap.release()
        cv2.destroyAllWindows()
        
        if len(baseline_landmarks_list) > 0 and len(baseline_normalized_list) > 0:
            # Calculate average baseline (both raw and normalized)
            self.baseline_landmarks = np.mean(baseline_landmarks_list, axis=0)
            self.baseline_normalized_landmarks = np.mean(baseline_normalized_list, axis=0)
            
            # Calculate average baseline face statistics
            if baseline_face_stats_list:
                avg_width = np.mean([stats[0] for stats in baseline_face_stats_list])
                avg_height = np.mean([stats[1] for stats in baseline_face_stats_list])
                avg_center = np.mean([stats[2] for stats in baseline_face_stats_list], axis=0)
                self.baseline_face_stats = (avg_width, avg_height, avg_center)
            
            print(f"✓ Baseline established from {len(baseline_landmarks_list)} frames")
            print(f"Raw baseline shape: {self.baseline_landmarks.shape}")
            print(f"Normalized baseline shape: {self.baseline_normalized_landmarks.shape}")
            
            # Debug baseline statistics
            if self.baseline_face_stats:
                print(f"Baseline face - Width: {self.baseline_face_stats[0]:.1f}, "
                      f"Height: {self.baseline_face_stats[1]:.1f}, "
                      f"Center: ({self.baseline_face_stats[2][0]:.1f},{self.baseline_face_stats[2][1]:.1f})")
            
            return True
        else:
            print(" Failed to establish baseline - no faces detected")
            return False

    def calculate_rmsd_deviation_detailed(self, current_data):
        """Calculate RMSD deviation using NORMALIZED landmarks - KEY FIX"""
        if self.baseline_normalized_landmarks is None or current_data is None:
            return 0.0, [], {}
        
        current_landmarks = current_data.get('landmarks_2d')
        if current_landmarks is None:
            return 0.0, [], {}
        
        # NORMALIZE current landmarks to handle position/scale differences
        current_normalized = self.normalize_landmarks_to_face_size(current_landmarks)
        if current_normalized is None:
            return 0.0, [], {}
        
        # Ensure shapes match
        if current_normalized.shape != self.baseline_normalized_landmarks.shape:
            print(f"  SHAPE MISMATCH: Baseline {self.baseline_normalized_landmarks.shape} vs Current {current_normalized.shape}")
            return 0.0, [], {}
        
        # Detailed calculation tracking
        detailed_calc = {
            'frame_number': self.frame_count,
            'timestamp': datetime.now().isoformat(),
            'normalization_applied': True,  # NEW: Track that normalization was used
            'formula_explanation': {
                'step1': 'Normalize landmarks to face-relative coordinates (0-1 range)',
                'step2': 'Calculate Euclidean distance for each normalized landmark: d(pf, pf+a) = sqrt((x_pf+a - x_pf)² + (y_pf+a - y_pf)²)',
                'step3': 'Square each distance: d²',
                'step4': 'Sum all squared distances: Σ(d²)',
                'step5': 'Divide by number of landmarks (N=68): Σ(d²)/N',
                'step6': 'Take square root for final RMSD: sqrt(Σ(d²)/N)'
            },
            'landmark_calculations': [],
            'intermediate_values': {}
        }
        
        # Calculate distances using NORMALIZED coordinates
        landmark_distances = []
        sum_squared_distances = 0.0
        
        for i in range(len(current_normalized)):
            baseline_point = self.baseline_normalized_landmarks[i]
            current_point = current_normalized[i]
            
            x_diff = current_point[0] - baseline_point[0]
            y_diff = current_point[1] - baseline_point[1]
            distance_squared = x_diff ** 2 + y_diff ** 2
            distance = np.sqrt(distance_squared)
            
            landmark_distances.append(distance)
            sum_squared_distances += distance_squared
            
            # Store detailed calculation for first 10 landmarks
            if i < 10:
                detailed_calc['landmark_calculations'].append({
                    'landmark_index': i,
                    'baseline_normalized': [float(baseline_point[0]), float(baseline_point[1])],
                    'current_normalized': [float(current_point[0]), float(current_point[1])],
                    'x_difference': float(x_diff),
                    'y_difference': float(y_diff),
                    'distance_squared': float(distance_squared),
                    'euclidean_distance': float(distance)
                })
        
        landmark_distances = np.array(landmark_distances)
        
        # Calculate RMSD
        N = len(landmark_distances)
        mean_squared_distance = sum_squared_distances / N
        rmsd = np.sqrt(mean_squared_distance)
        
        # Store intermediate values
        detailed_calc['intermediate_values'] = {
            'N_landmarks': N,
            'sum_squared_distances': float(sum_squared_distances),
            'mean_squared_distance': float(mean_squared_distance),
            'rmsd_final': float(rmsd)
        }
        
        # Add statistics
        detailed_calc['landmark_statistics'] = {
            'min_distance': float(np.min(landmark_distances)),
            'max_distance': float(np.max(landmark_distances)),
            'mean_distance': float(np.mean(landmark_distances)),
            'std_distance': float(np.std(landmark_distances)),
            'median_distance': float(np.median(landmark_distances))
        }
        
        return float(rmsd), landmark_distances.tolist(), detailed_calc

    def debug_landmark_distances(self, current_data):
        """Enhanced debug method using normalized coordinates"""
        if self.baseline_normalized_landmarks is None or current_data is None:
            return
        
        current_landmarks = current_data.get('landmarks_2d')
        if current_landmarks is None:
            return
        
        current_normalized = self.normalize_landmarks_to_face_size(current_landmarks)
        if current_normalized is None:
            return
        
        print(f"\n=== DEBUGGING FRAME {self.frame_count} (NORMALIZED COORDINATES) ===")
        print(f"Baseline normalized shape: {self.baseline_normalized_landmarks.shape}")
        print(f"Current normalized shape: {current_normalized.shape}")
        
        # Check face consistency (raw coordinates)
        current_face_stats = self.check_face_consistency(current_landmarks)
        if current_face_stats and self.baseline_face_stats:
            print(f"Raw baseline face - W:{self.baseline_face_stats[0]:.1f} H:{self.baseline_face_stats[1]:.1f}")
            print(f"Raw current face  - W:{current_face_stats[0]:.1f} H:{current_face_stats[1]:.1f}")
            
            # Check for significant face size changes
            width_change = abs(current_face_stats[0] - self.baseline_face_stats[0]) / self.baseline_face_stats[0] * 100
            height_change = abs(current_face_stats[1] - self.baseline_face_stats[1]) / self.baseline_face_stats[1] * 100
            
            if width_change > 20 or height_change > 20:
                print(f"  RAW FACE SIZE MISMATCH: Width change {width_change:.1f}%, Height change {height_change:.1f}%")
            else:
                print(f" Face size consistent: Width change {width_change:.1f}%, Height change {height_change:.1f}%")
        
        # Check normalized coordinates (should be much more consistent)
        print(f"\nNORMALIZED COORDINATE ANALYSIS:")
        
        # Check first 5 landmarks in detail
        for i in range(min(5, len(current_normalized))):
            baseline_point = self.baseline_normalized_landmarks[i]
            current_point = current_normalized[i]
            
            x_diff = current_point[0] - baseline_point[0]
            y_diff = current_point[1] - baseline_point[1]
            distance = np.sqrt(x_diff**2 + y_diff**2)
            
            print(f"Landmark {i:2d}: Base({baseline_point[0]:6.3f},{baseline_point[1]:6.3f}) "
                  f"Curr({current_point[0]:6.3f},{current_point[1]:6.3f}) "
                  f"Diff({x_diff:6.3f},{y_diff:6.3f}) Dist={distance:.3f}")
        
        # Overall normalized statistics
        all_distances = []
        for i in range(len(current_normalized)):
            baseline_point = self.baseline_normalized_landmarks[i]
            current_point = current_normalized[i]
            distance = np.sqrt((current_point[0] - baseline_point[0])**2 +
                             (current_point[1] - baseline_point[1])**2)
            all_distances.append(distance)
        
        print(f"Normalized distance stats - Min: {np.min(all_distances):.4f}, "
              f"Max: {np.max(all_distances):.4f}, "
              f"Mean: {np.mean(all_distances):.4f}, "
              f"Median: {np.median(all_distances):.4f}")
        
        # Flag if still getting extreme values in normalized space
        if np.mean(all_distances) > 0.1:  # 0.1 is 10% of face in normalized space
            print(f"  HIGH NORMALIZED DISTANCES - May indicate expression change or detection error")
        else:
            print(f" Normalized distances look reasonable")
        
        print("="*70)

    def debug_baseline_vs_current(self, current_data, frame):
        """Debug visualization using both raw and normalized coordinates"""
        if self.baseline_landmarks is None or current_data is None:
            return frame
        
        current_landmarks = current_data.get('landmarks_2d')
        if current_landmarks is None:
            return frame
        
        # Only show debug visualization if debug_mode is on
        if not self.debug_mode:
            return frame
        
        # Draw baseline landmarks in blue (smaller circles)
        for i, point in enumerate(self.baseline_landmarks):
            x, y = int(point[0]), int(point[1])
            if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0]:
                cv2.circle(frame, (x, y), 1, (255, 0, 0), -1)  # Blue for baseline
        
        # Draw current landmarks in red
        for i, point in enumerate(current_landmarks):
            x, y = int(point[0]), int(point[1])
            if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0]:
                cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)  # Red for current
                
                # Draw line connecting baseline to current for high deviation landmarks
                baseline_point = self.baseline_landmarks[i]
                bx, by = int(baseline_point[0]), int(baseline_point[1])
                raw_distance = np.sqrt((x - bx)**2 + (y - by)**2)
                
                # Only draw connection lines for high deviation points in raw space
                if raw_distance > 10:  # Show connections for points that moved >10 pixels
                    cv2.line(frame, (bx, by), (x, y), (0, 255, 0), 1)  # Green connecting line
        
        # Add debug info with normalized distance information
        cv2.putText(frame, "DEBUG: Blue=Baseline, Red=Current, Green=High Raw Deviation",
                   (10, frame.shape[0] - 70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        cv2.putText(frame, "RMSD calculated using NORMALIZED coordinates",
                   (10, frame.shape[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1)
        
        return frame

    def start_tracking(self, duration_minutes=None, jupyter_mode=False):
        """Start real-time tracking - asks user for duration or runs until 'q' is pressed"""
        if self.baseline_normalized_landmarks is None:
            print(" Baseline not established. Run setup_baseline() first.")
            return
        
        # Handle user input for tracking duration
        if duration_minutes is None and not jupyter_mode:
            print("\n Starting real-time RMSD tracking...")
            while True:
                try:
                    user_input = input("Enter tracking duration in minutes (or press Enter for unlimited): ").strip()
                    if user_input == "":
                        duration_minutes = None
                        print(" Unlimited tracking mode - press 'q' to quit anytime")
                        break
                    else:
                        duration_minutes = float(user_input)
                        if duration_minutes > 0:
                            print(f" Tracking for {duration_minutes} minutes (press 'q' to quit early)")
                            break
                        else:
                            print("Please enter a positive number")
                except ValueError:
                    print("Please enter a valid number or press Enter for unlimited")
        elif duration_minutes is None and jupyter_mode:
            print("\n Jupyter mode: Running unlimited tracking - press 'q' to quit")
            duration_minutes = None
        elif duration_minutes is not None:
            print(f"\n Starting {duration_minutes}-minute tracking session")
        
        print(" Using NORMALIZED coordinates to handle position/scale variations")
        print("DEBUG MODE ENABLED - Extra diagnostic information will be shown")
        print("Press 'q' to quit, 's' to save data, 'd' to toggle debug mode")
        
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            raise Exception("Could not open webcam")
        
        # Set same camera parameters as baseline
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        start_time = time.time()
        end_time = start_time + (duration_minutes * 60) if duration_minutes else None
        
        while True:
            # Check if duration limit reached (only if duration_minutes is specified)
            if end_time and time.time() >= end_time:
                print(f"\n Time limit of {duration_minutes} minutes reached")
                break
            ret, frame = cap.read()
            if not ret:
                continue
            
            temp_image = os.path.join(self.temp_dir, f"frame_{self.frame_count}.jpg")
            cv2.imwrite(temp_image, frame)
            
            current_data = self.extract_landmarks_from_image(temp_image)
            
            if current_data is not None:
                # Debug output for first 3 frames
                if self.frame_count < 3 and self.debug_mode:
                    self.debug_landmark_distances(current_data)
                
                # Calculate RMSD with normalization
                rmsd_deviation, landmark_distances, detailed_calc = self.calculate_rmsd_deviation_detailed(current_data)
                self.accumulated_rmsd += rmsd_deviation
                
                self.detailed_calculations.append(detailed_calc)
                
                # Store data
                timestamp = datetime.now()
                self.deviation_data.append({
                    'timestamp': timestamp.isoformat(),
                    'frame': self.frame_count,
                    'rmsd_deviation': float(rmsd_deviation),
                    'accumulated_rmsd': float(self.accumulated_rmsd),
                    'avg_rmsd': float(self.accumulated_rmsd / (self.frame_count + 1)),
                    'max_landmark_deviation': float(max(landmark_distances)) if landmark_distances else 0.0,
                    'min_landmark_deviation': float(min(landmark_distances)) if landmark_distances else 0.0,
                    'std_landmark_deviation': float(np.std(landmark_distances)) if landmark_distances else 0.0,
                    'normalization_used': True  # Track that normalization was applied
                })
                
                # Add debug visualization
                if self.debug_mode:
                    frame = self.debug_baseline_vs_current(current_data, frame)
                
                # Get visualization data
                landmarks_2d = current_data.get('landmarks_2d')
                head_pose = current_data.get('head_pose')
                left_eye = current_data.get('left_eye')
                right_eye = current_data.get('right_eye')
                gaze = current_data.get('gaze')
                
                if landmarks_2d is not None:
                    face_center = np.mean(landmarks_2d, axis=0)
                    
                    # Draw visualizations (only if not in debug mode to avoid clutter)
                    if not self.debug_mode:
                        if head_pose is not None:
                            frame = self.draw_openface_3d_cube(frame, head_pose, face_center)
                        frame = self.draw_landmark_deviation_visualization(frame, landmarks_2d, landmark_distances)
                        frame = self.draw_openface_landmarks_and_gaze(frame, landmarks_2d, left_eye, right_eye, gaze)
                
                # Display tracking information with improved color coding
                elapsed = time.time() - start_time
                
                # Calculate remaining time only if duration is set
                if end_time:
                    remaining = max(0, end_time - time.time())
                    time_display = f"Time: {int(elapsed)}s / {int(remaining)}s left"
                else:
                    hours = int(elapsed // 3600)
                    minutes = int((elapsed % 3600) // 60)
                    seconds = int(elapsed % 60)
                    if hours > 0:
                        time_display = f"Time: {hours}h {minutes}m {seconds}s"
                    elif minutes > 0:
                        time_display = f"Time: {minutes}m {seconds}s"
                    else:
                        time_display = f"Time: {seconds}s"
                
                # Info panel with subtle background and border
                panel_height = 320 if self.debug_mode else 270
                cv2.rectangle(frame, (10, 10), (550, panel_height), (40, 40, 40), -1)  # Dark background
                cv2.rectangle(frame, (10, 10), (550, panel_height), (100, 150, 200), 2)  # Blue border
                
                # Header section
                cv2.rectangle(frame, (15, 15), (545, 45), (60, 60, 60), -1)  # Header background
                cv2.putText(frame, "REAL-TIME FACIAL DEVIATION ANALYSIS", (20, 35),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # RMSD section with color-coded background
                rmsd_bg_color = (0, 80, 0) if rmsd_deviation < 0.01 else (0, 60, 80) if rmsd_deviation < 0.05 else (0, 0, 80)
                cv2.rectangle(frame, (15, 50), (545, 125), rmsd_bg_color, -1)  # RMSD section background
                
                # RMSD info with realistic color coding for normalized values
                if rmsd_deviation < 0.01:
                    rmsd_color = (100, 255, 100)    # Light green - very small deviation
                    status_text = "NORMAL"
                    status_color = (100, 255, 100)
                elif rmsd_deviation < 0.05:
                    rmsd_color = (0, 200, 255)      # Light orange - moderate deviation
                    status_text = "ELEVATED"
                    status_color = (0, 200, 255)
                else:
                    rmsd_color = (100, 100, 255)    # Light red - high deviation
                    status_text = "HIGH"
                    status_color = (100, 100, 255)
                
                cv2.putText(frame, f"Normalized RMSD: {rmsd_deviation:.6f}", (20, 70),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, rmsd_color, 2)
                cv2.putText(frame, f"Accumulated RMSD: {self.accumulated_rmsd:.6f}", (20, 90),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
                cv2.putText(frame, f"Average RMSD: {self.accumulated_rmsd / (self.frame_count + 1):.6f}", (20, 110),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
                
                # Status indicator with background
                cv2.rectangle(frame, (350, 55), (540, 85), (80, 80, 80), -1)  # Status background
                cv2.putText(frame, f"Status: {status_text}", (355, 75),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)
                
                # Statistics section
                cv2.rectangle(frame, (15, 130), (545, 190), (50, 50, 50), -1)  # Stats background
                cv2.putText(frame, f"Max Normalized Dev: {max(landmark_distances) if landmark_distances else 0:.4f}", (20, 150),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 255, 255), 1)
                cv2.putText(frame, time_display, (20, 170),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (150, 255, 150), 2)
                cv2.putText(frame, f"Frames Processed: {len(self.deviation_data)}", (280, 170),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150, 255, 150), 1)
                
                # Head pose section
                if head_pose is not None and len(head_pose) > 0:
                    cv2.rectangle(frame, (15, 195), (545, 235), (60, 40, 60), -1)  # Pose background
                    pitch, yaw, roll = head_pose * 180 / np.pi
                    cv2.putText(frame, f"Head Pose - Pitch: {pitch:.1f}deg", (20, 215),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 180, 100), 1)
                    cv2.putText(frame, f"Yaw: {yaw:.1f}deg  Roll: {roll:.1f}deg", (20, 230),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 180, 100), 1)
                    controls_y = 245
                else:
                    controls_y = 200
                
                # Control indicators section
                cv2.rectangle(frame, (15, controls_y), (545, controls_y + 60), (40, 60, 40), -1)  # Controls background
                cv2.putText(frame, "NORMALIZATION ACTIVE", (20, controls_y + 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 255, 100), 1)
                
                # Control instructions
                cv2.putText(frame, "Controls: Q=Quit | S=Save | D=Debug Toggle", (20, controls_y + 40),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 100), 1)
                
                # Debug mode indicator
                if self.debug_mode:
                    cv2.putText(frame, "DEBUG MODE ACTIVE", (20, controls_y + 55),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 255, 255), 1)
                
            else:
                # No face detected
                cv2.rectangle(frame, (10, 10), (300, 80), (0, 0, 0), -1)
                cv2.putText(frame, "No face detected", (15, 35),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "Ensure good lighting and face camera", (15, 60),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
            
            cv2.putText(frame, "OpenFace RMSD Tracking (Normalized Coordinates)", (10, frame.shape[0] - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow('OpenFace RMSD Facial Deviation Tracking', frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("\n Tracking stopped by user")
                break
            elif key == ord('s'):
                csv_file, report_file = self.save_all_data()
                print(f" Data saved! CSV: {csv_file}, Report: {report_file}")
            elif key == ord('d'):
                self.debug_mode = not self.debug_mode
                print(f" Debug mode: {'ON' if self.debug_mode else 'OFF'}")
            
            self.frame_count += 1
            
            # Clean up frame file
            if os.path.exists(temp_image):
                os.remove(temp_image)
        
        cap.release()
        cv2.destroyAllWindows()
        
        print(f"\n Tracking completed! Processed {len(self.deviation_data)} frames")
        return self.save_all_data()

    def extract_landmarks_from_image(self, image_path):
        """Extract facial landmarks and pose data from a single image using OpenFace"""
        try:
            output_dir = os.path.join(self.temp_dir, f"output_{int(time.time() * 1000)}")
            os.makedirs(output_dir, exist_ok=True)
            
            cmd = [
                self.feature_extraction,
                "-f", image_path,
                "-out_dir", output_dir,
                "-2Dfp", "-3Dfp", "-pdmparams", "-pose", "-gaze", "-aus",
                "-quiet"
            ]
            
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
            
            if result.returncode != 0:
                print(f"OpenFace error: {result.stderr}")
                return None
            
            csv_files = [f for f in os.listdir(output_dir) if f.endswith('.csv')]
            if not csv_files:
                return None
            
            csv_path = os.path.join(output_dir, csv_files[0])
            
            try:
                df = pd.read_csv(csv_path, engine='python')
            except:
                with open(csv_path, 'r') as f:
                    lines = f.readlines()
                    if len(lines) < 2:
                        return None
                    
                    headers = lines[0].strip().split(',')
                    values = lines[1].strip().split(',')
                    
                    data_dict = {}
                    for i, header in enumerate(headers):
                        if i < len(values):
                            try:
                                data_dict[header] = float(values[i])
                            except ValueError:
                                data_dict[header] = values[i]
                    
                    df = pd.DataFrame([data_dict])
            
            data = {}
            
            # Extract 2D landmarks
            landmark_cols = []
            for i in range(68):
                x_col = f'x_{i}'
                y_col = f'y_{i}'
                if x_col in df.columns and y_col in df.columns:
                    landmark_cols.extend([x_col, y_col])
            
            if landmark_cols:
                landmarks = df[landmark_cols].iloc[0].values
                data['landmarks_2d'] = landmarks.reshape(-1, 2)
            
            # Extract head pose (pitch, yaw, roll)
            pose_cols = ['pose_Rx', 'pose_Ry', 'pose_Rz']
            if all(col in df.columns for col in pose_cols):
                data['head_pose'] = df[pose_cols].iloc[0].values
            
            # Extract gaze direction
            gaze_cols = ['gaze_0_x', 'gaze_0_y', 'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z']
            available_gaze_cols = [col for col in gaze_cols if col in df.columns]
            if available_gaze_cols:
                data['gaze'] = df[available_gaze_cols].iloc[0].values
            
            # Extract eye landmarks for gaze visualization
            if 'landmarks_2d' in data:
                landmarks_2d = data['landmarks_2d']
                left_eye_indices = list(range(36, 42))
                right_eye_indices = list(range(42, 48))
                
                data['left_eye'] = landmarks_2d[left_eye_indices]
                data['right_eye'] = landmarks_2d[right_eye_indices]
            
            shutil.rmtree(output_dir, ignore_errors=True)
            return data if data else None
            
        except Exception as e:
            print(f"Error extracting landmarks: {e}")
            return None

    def check_face_consistency(self, landmarks):
        """Check face bounding box consistency for debugging"""
        if landmarks is not None and len(landmarks) > 0:
            face_width = np.max(landmarks[:,0]) - np.min(landmarks[:,0])
            face_height = np.max(landmarks[:,1]) - np.min(landmarks[:,1])
            face_center = np.mean(landmarks, axis=0)
            
            return face_width, face_height, face_center
        return None

    def draw_openface_3d_cube(self, frame, head_pose, face_center):
        """Draw 3D head pose cube exactly like OpenFace"""
        if head_pose is None or len(head_pose) != 3:
            return frame
        
        pitch, yaw, roll = head_pose
        cube_size = 80
        cube_3d = np.array([
            [-cube_size, -cube_size, -cube_size], [cube_size, -cube_size, -cube_size],
            [cube_size, cube_size, -cube_size], [-cube_size, cube_size, -cube_size],
            [-cube_size, -cube_size, cube_size], [cube_size, -cube_size, cube_size],
            [cube_size, cube_size, cube_size], [-cube_size, cube_size, cube_size]
        ], dtype=np.float32)
        
        # Rotation matrices
        rx = np.array([[1, 0, 0], [0, np.cos(pitch), -np.sin(pitch)], [0, np.sin(pitch), np.cos(pitch)]])
        ry = np.array([[np.cos(yaw), 0, np.sin(yaw)], [0, 1, 0], [-np.sin(yaw), 0, np.cos(yaw)]])
        rz = np.array([[np.cos(roll), -np.sin(roll), 0], [np.sin(roll), np.cos(roll), 0], [0, 0, 1]])
        
        rotation_matrix = rz @ ry @ rx
        rotated_cube = cube_3d @ rotation_matrix.T
        
        # Perspective projection
        focal_length = 500
        projected_points = []
        for point in rotated_cube:
            x, y, z = point
            x += face_center[0]
            y += face_center[1]
            z += 200
            
            if z != 0:
                proj_x = int(focal_length * x / z + face_center[0])
                proj_y = int(focal_length * y / z + face_center[1])
            else:
                proj_x = int(x + face_center[0])
                proj_y = int(y + face_center[1])
            
            projected_points.append((proj_x, proj_y))
        
        projected_points = np.array(projected_points)
        
        # Draw cube edges
        edges = [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (0, 4), (1, 5), (2, 6), (3, 7)]
        openface_blue = (255, 128, 0)
        
        for edge in edges:
            pt1 = tuple(projected_points[edge[0]])
            pt2 = tuple(projected_points[edge[1]])
            
            if (0 <= pt1[0] < frame.shape[1] and 0 <= pt1[1] < frame.shape[0] and
                0 <= pt2[0] < frame.shape[1] and 0 <= pt2[1] < frame.shape[0]):
                cv2.line(frame, pt1, pt2, openface_blue, 2)
        
        return frame

    def draw_landmark_deviation_visualization(self, frame, landmarks_2d, landmark_distances):
        """Draw landmarks with color-coded deviation intensity"""
        if landmarks_2d is None or not landmark_distances:
            return frame
        
        max_distance = max(landmark_distances) if landmark_distances else 1
        
        for i, (point, distance) in enumerate(zip(landmarks_2d, landmark_distances)):
            x, y = int(point[0]), int(point[1])
            if 0 <= x < frame.shape[1] and 0 <= y < frame.shape[0]:
                intensity = min(255, int((distance / max_distance) * 255))
                color = (0, 255 - intensity, intensity)
                cv2.circle(frame, (x, y), 3, color, -1)
                
                if distance > max_distance * 0.7:
                    cv2.circle(frame, (x, y), 6, (0, 0, 255), 2)
        
        return frame

    def draw_openface_landmarks_and_gaze(self, frame, landmarks_2d, left_eye, right_eye, gaze=None):
        """Draw facial landmarks and gaze exactly like OpenFace"""
        if landmarks_2d is None:
            return frame
        
        if left_eye is not None and right_eye is not None:
            left_center = np.mean(left_eye, axis=0).astype(int)
            right_center = np.mean(right_eye, axis=0).astype(int)
            
            eye_radius = 15
            cv2.circle(frame, tuple(left_center), eye_radius, (0, 255, 0), 2)
            cv2.circle(frame, tuple(right_center), eye_radius, (0, 255, 0), 2)
            
            if gaze is not None and len(gaze) >= 6:
                gaze_scale = 40
                left_gaze_end = (int(left_center[0] + gaze[0] * gaze_scale), int(left_center[1] + gaze[1] * gaze_scale))
                right_gaze_end = (int(right_center[0] + gaze[3] * gaze_scale), int(right_center[1] + gaze[4] * gaze_scale))
                cv2.arrowedLine(frame, tuple(left_center), left_gaze_end, (255, 0, 255), 2)
                cv2.arrowedLine(frame, tuple(right_center), right_gaze_end, (255, 0, 255), 2)
        
        return frame

    def save_all_data(self, base_filename=None):
        """Save both CSV data and detailed calculation report to Desktop/Facial Deviation Work folder"""
        if not self.deviation_data:
            print("No data to save")
            return None, None
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Create "Facial Deviation Work" folder on Desktop
        home_dir = os.path.expanduser("~")
        desktop_path = os.path.join(home_dir, "Desktop")
        save_directory = os.path.join(desktop_path, "Facial Deviation Work")
        
        try:
            os.makedirs(save_directory, exist_ok=True)
            test_file = os.path.join(save_directory, "test_write.tmp")
            with open(test_file, 'w') as f:
                f.write("test")
            os.remove(test_file)
            print(f"Saving files to: {save_directory}")
        except (OSError, PermissionError) as e:
            print(f"Could not create/access Desktop folder: {e}")
            save_directory = os.path.join(home_dir, "Facial Deviation Work")
            try:
                os.makedirs(save_directory, exist_ok=True)
                print(f"Using fallback directory: {save_directory}")
            except:
                print("Could not create writable directory for saving files")
                return None, None
        
        if base_filename is None:
            csv_filename = os.path.join(save_directory, f"facial_rmsd_normalized_{timestamp}.csv")
            report_filename = os.path.join(save_directory, f"facial_rmsd_normalized_report_{timestamp}.json")
        else:
            csv_filename = os.path.join(save_directory, f"{base_filename}_{timestamp}.csv")
            report_filename = os.path.join(save_directory, f"{base_filename}_detailed_report_{timestamp}.json")
        
        # Save CSV data
        try:
            with open(csv_filename, 'w', newline='') as csvfile:
                if self.deviation_data:
                    headers = list(self.deviation_data[0].keys())
                    csvfile.write(','.join(headers) + '\n')
                    
                    for row in self.deviation_data:
                        values = [str(row[key]) for key in headers]
                        csvfile.write(','.join(values) + '\n')
                        
            print(f"CSV data saved to: {csv_filename}")
        except Exception as e:
            print(f"Failed to save CSV data: {e}")
            return None, None
        
        # Save detailed report
        try:
            report_data = {
                'session_info': {
                    'timestamp': timestamp,
                    'total_frames': len(self.deviation_data),
                    'total_accumulated_rmsd': float(self.accumulated_rmsd),
                    'session_duration': f"{len(self.deviation_data)} frames",
                    'save_directory': save_directory,
                    'normalization_applied': True,
                    'rmsd_formula': {
                        'description': 'Root Mean Square Deviation from normalized baseline landmarks',
                        'formula': 'RMSD = sqrt(Σ(d(pf, pf+a)²) / N)',
                        'where': 'd(pf, pf+a) = sqrt((x_pf+a - x_pf)² + (y_pf+a - y_pf)²)',
                        'N': '68 facial landmarks',
                        'coordinates': 'Normalized to face-relative coordinates (0-1 range)'
                    }
                },
                'summary_statistics': self._calculate_summary_statistics(),
                'detailed_frame_calculations': self.detailed_calculations
            }
            
            with open(report_filename, 'w') as jsonfile:
                json.dump(report_data, jsonfile, indent=2)
                
            print(f"Detailed report saved to: {report_filename}")
        except Exception as e:
            print(f"Failed to save detailed report: {e}")
            return csv_filename, None
        
        self._display_summary_statistics()
        return csv_filename, report_filename

    def _calculate_summary_statistics(self):
        """Calculate comprehensive summary statistics"""
        if not self.deviation_data:
            return {}
        
        rmsd_values = []
        max_landmark_devs = []
        min_landmark_devs = []
        std_landmark_devs = []
        
        for data_point in self.deviation_data:
            if 'rmsd_deviation' in data_point:
                rmsd_values.append(float(data_point['rmsd_deviation']))
            if 'max_landmark_deviation' in data_point:
                max_landmark_devs.append(float(data_point['max_landmark_deviation']))
            if 'min_landmark_deviation' in data_point:
                min_landmark_devs.append(float(data_point['min_landmark_deviation']))
            if 'std_landmark_deviation' in data_point:
                std_landmark_devs.append(float(data_point['std_landmark_deviation']))
        
        summary = {
            'rmsd_statistics': {
                'count': len(rmsd_values),
                'mean': float(np.mean(rmsd_values)) if rmsd_values else 0.0,
                'median': float(np.median(rmsd_values)) if rmsd_values else 0.0,
                'std': float(np.std(rmsd_values)) if rmsd_values else 0.0,
                'min': float(np.min(rmsd_values)) if rmsd_values else 0.0,
                'max': float(np.max(rmsd_values)) if rmsd_values else 0.0,
                'percentiles': {
                    '25th': float(np.percentile(rmsd_values, 25)) if rmsd_values else 0.0,
                    '75th': float(np.percentile(rmsd_values, 75)) if rmsd_values else 0.0,
                    '90th': float(np.percentile(rmsd_values, 90)) if rmsd_values else 0.0,
                    '95th': float(np.percentile(rmsd_values, 95)) if rmsd_values else 0.0
                }
            },
            'landmark_deviation_statistics': {
                'max_deviations': {
                    'mean': float(np.mean(max_landmark_devs)) if max_landmark_devs else 0.0,
                    'max': float(np.max(max_landmark_devs)) if max_landmark_devs else 0.0,
                    'min': float(np.min(max_landmark_devs)) if max_landmark_devs else 0.0
                },
                'min_deviations': {
                    'mean': float(np.mean(min_landmark_devs)) if min_landmark_devs else 0.0,
                    'max': float(np.max(min_landmark_devs)) if min_landmark_devs else 0.0,
                    'min': float(np.min(min_landmark_devs)) if min_landmark_devs else 0.0
                }
            },
            'trend_analysis': self._analyze_trends(rmsd_values) if rmsd_values else {},
            'high_deviation_analysis': self._analyze_high_deviations(rmsd_values) if rmsd_values else {}
        }
        return summary

    def _analyze_trends(self, rmsd_values):
        """Analyze trends in RMSD values over time"""
        if len(rmsd_values) < 10:
            return {'note': 'Insufficient data for trend analysis (need at least 10 frames)'}
        
        segment_size = len(rmsd_values) // 3
        first_third = rmsd_values[:segment_size]
        middle_third = rmsd_values[segment_size:2*segment_size]
        last_third = rmsd_values[2*segment_size:]
        
        first_mean = np.mean(first_third)
        middle_mean = np.mean(middle_third)
        last_mean = np.mean(last_third)
        
        if last_mean > first_mean * 1.1:
            overall_trend = "increasing"
        elif last_mean < first_mean * 0.9:
            overall_trend = "decreasing"
        else:
            overall_trend = "stable"
        
        return {
            'overall_trend': overall_trend,
            'first_third_mean': float(first_mean),
            'middle_third_mean': float(middle_mean),
            'last_third_mean': float(last_mean),
            'trend_magnitude': float(abs(last_mean - first_mean)),
            'trend_percentage': float(((last_mean - first_mean) / first_mean) * 100) if first_mean > 0 else 0.0
        }

    def _analyze_high_deviations(self, rmsd_values):
        """Analyze frames with high deviations"""
        if not rmsd_values:
            return {}
        
        threshold_90 = np.percentile(rmsd_values, 90)
        threshold_95 = np.percentile(rmsd_values, 95)
        
        high_dev_90 = [i for i, val in enumerate(rmsd_values) if val > threshold_90]
        high_dev_95 = [i for i, val in enumerate(rmsd_values) if val > threshold_95]
        
        return {
            'threshold_90th_percentile': float(threshold_90),
            'threshold_95th_percentile': float(threshold_95),
            'frames_above_90th': len(high_dev_90),
            'frames_above_95th': len(high_dev_95),
            'high_deviation_frame_indices_90th': high_dev_90[:10],
            'high_deviation_frame_indices_95th': high_dev_95[:10],
            'percentage_high_deviation_frames': float((len(high_dev_90) / len(rmsd_values)) * 100)
        }

    def _display_summary_statistics(self):
        """Display summary statistics in console"""
        if not self.deviation_data:
            print("No data available for summary")
            return
        
        summary = self._calculate_summary_statistics()
        
        print("\n NORMALIZED RMSD ANALYSIS SUMMARY:")
        print("=" * 50)
        
        rmsd_stats = summary.get('rmsd_statistics', {})
        if rmsd_stats:
            print(f" RMSD Statistics ({rmsd_stats['count']} frames):")
            print(f"  • Mean RMSD: {rmsd_stats['mean']:.8f}")
            print(f"  • Median RMSD: {rmsd_stats['median']:.8f}")
            print(f"  • Standard Deviation: {rmsd_stats['std']:.8f}")
            print(f"  • Range: {rmsd_stats['min']:.8f} - {rmsd_stats['max']:.8f}")
            print(f"  • 90th Percentile: {rmsd_stats['percentiles']['90th']:.8f}")
            print(f"  • 95th Percentile: {rmsd_stats['percentiles']['95th']:.8f}")
        
        trend_analysis = summary.get('trend_analysis', {})
        if trend_analysis and 'overall_trend' in trend_analysis:
            print(f"\n Trend Analysis:")
            print(f"  • Overall trend: {trend_analysis['overall_trend']}")
            print(f"  • Trend magnitude: {trend_analysis['trend_magnitude']:.8f}")
            print(f"  • Percentage change: {trend_analysis['trend_percentage']:.2f}%")
        
        high_dev = summary.get('high_deviation_analysis', {})
        if high_dev:
            print(f"\n High Deviation Analysis:")
            print(f"  • Frames above 90th percentile: {high_dev['frames_above_90th']}")
            print(f"  • Frames above 95th percentile: {high_dev['frames_above_95th']}")
            print(f"  • Percentage of high deviation frames: {high_dev['percentage_high_deviation_frames']:.2f}%")
        
        print("\n Key Improvements in This Version:")
        print("  • COORDINATE NORMALIZATION - Fixed position/scale issues")
        print("  • Realistic RMSD values (0.001-0.1 range)")
        print("  • Face-relative landmark comparison")
        print("  • Enhanced debugging with normalized coordinates")

    def analyze_deviation_patterns(self):
        """Analyze deviation patterns and provide insights"""
        if not self.deviation_data:
            print("No data available for analysis")
            return
        
        print("\n NORMALIZED RMSD Deviation Pattern Analysis:")
        
        summary = self._calculate_summary_statistics()
        rmsd_stats = summary.get('rmsd_statistics', {})
        
        if rmsd_stats:
            print(f"  • Average RMSD: {rmsd_stats['mean']:.8f}")
            print(f"  • RMSD Range: {rmsd_stats['min']:.8f} - {rmsd_stats['max']:.8f}")
            print(f"  • RMSD Standard Deviation: {rmsd_stats['std']:.8f}")
            
            trend_analysis = summary.get('trend_analysis', {})
            if trend_analysis and 'overall_trend' in trend_analysis:
                print(f"  • Overall trend: {trend_analysis['overall_trend']}")
                print(f"  • Trend change: {trend_analysis['trend_percentage']:.2f}%")
            
            high_dev = summary.get('high_deviation_analysis', {})
            if high_dev:
                print(f"  • High deviation frames (top 10%): {high_dev['frames_above_90th']} frames")
                print(f"  • Extreme deviation frames (top 5%): {high_dev['frames_above_95th']} frames")
        else:
            print("  • Error: Could not calculate statistics")
            print(f"  • Raw data points: {len(self.deviation_data)}")
            print(f"  • Accumulated RMSD: {self.accumulated_rmsd:.8f}")

    def cleanup(self):
        """Clean up temporary files"""
        if hasattr(self, 'temp_dir') and os.path.exists(self.temp_dir):
            shutil.rmtree(self.temp_dir, ignore_errors=True)
            print(f"Cleaned up temporary directory: {self.temp_dir}")
        else:
            print("No temporary directory to clean up")

# Usage Example with Enhanced Debugging
if __name__ == "__main__":
    csv_file = None
    report_file = None
    
    try:
        print("=== INITIALIZATION ===")
        tracker = RealTimeFacialDeviationTracker()
        
        print("=== DIAGNOSTIC CHECKS ===")
        import os
        print(f"Current working directory: {os.getcwd()}")
        print(f"Home directory: {os.path.expanduser('~')}")
        desktop_work_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Facial Deviation Work")
        try:
            os.makedirs(desktop_work_dir, exist_ok=True)
            print(f"Successfully created/verified: {desktop_work_dir}")
        except Exception as e:
            print(f"Cannot create Desktop folder: {e}")
        
        print("\n=== STEP 1: BASELINE SETUP ===")
        if tracker.setup_baseline(num_frames=20):
            print("Baseline established successfully!")
            
            print("\n=== STEP 2: REAL-TIME TRACKING ===")
            print("DEBUG MODE: Will show detailed diagnostics for first few frames")
            csv_file, report_file = tracker.start_tracking(jupyter_mode=True)
            
            if csv_file and report_file:
                print(f"\nSession complete!")
                print(f"CSV Data: {csv_file}")
                print(f"Detailed Report: {report_file}")
                
                print("\n=== STEP 3: PATTERN ANALYSIS ===")
                tracker.analyze_deviation_patterns()
                
                print(f"\nFiles Created:")
                print(f"  1. {csv_file} - Raw RMSD data")
                print(f"  2. {report_file} - Detailed calculations and diagnostics")
            else:
                print("Error occurred during data saving")
        else:
            print("Failed to establish baseline")
            print("Tips:")
            print("  • Ensure good lighting")
            print("  • Face the camera directly")
            print("  • Keep head still during baseline")
    
    except KeyboardInterrupt:
        print("\nInterrupted by user")
        if 'tracker' in locals() and tracker.deviation_data:
            print("Attempting to save partial data...")
            csv_file, report_file = tracker.save_all_data()
            if csv_file:
                print(f"Partial data saved: {csv_file}")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if 'tracker' in locals():
            tracker.cleanup()
        print("Session ended")